In [ ]:
import random
import numpy as np

# Genetic Algorithm for Travelling Salesman Problem (TSP)
def tsp_genetic_algorithm(cost_matrix, population_size=100, generations=500, mutation_rate=0.01):
    n = len(cost_matrix)

    def create_individual():
        return list(np.random.permutation(n))

    def fitness(individual):
        return sum(cost_matrix[individual[i]][individual[i + 1]] for i in range(n - 1)) + cost_matrix[individual[-1]][individual[0]]

    def crossover(parent1, parent2):
        cut = random.randint(1, n - 2)
        child = parent1[:cut] + [gene for gene in parent2 if gene not in parent1[:cut]]
        return child

    def mutate(individual):
        if random.random() < mutation_rate:
            i, j = random.sample(range(n), 2)
            individual[i], individual[j] = individual[j], individual[i]

    def select(population):
        return min(population, key=fitness)

    population = [create_individual() for _ in range(population_size)]

    for _ in range(generations):
        population = [select([crossover(random.choice(population), random.choice(population)) for _ in range(population_size)]) for _ in range(population_size)]
        for individual in population:
            mutate(individual)

    best_individual = select(population)
    return best_individual, fitness(best_individual)

# Genetic Algorithm for Optimal Power Flow (OPF)
def opf_genetic_algorithm(m, PD, generators, population_size=100, generations=500, mutation_rate=0.01, penalty_factor=1e6):
    def create_individual():
        return [random.uniform(gen[3], gen[4]) for gen in generators]
    
    def fitness(individual):
        cost = sum(gen[0] + gen[1] * individual[i] + gen[2] * individual[i] ** 2 for i, gen in enumerate(generators))
        power_balance_violation = abs(sum(individual) - (PD + 0.05 * PD))
        if any(ind < generators[i][3] or ind > generators[i][4] for i, ind in enumerate(individual)):
            return cost + penalty_factor * power_balance_violation
        return cost

    def crossover(parent1, parent2):
        cut = random.randint(1, m - 1)
        return parent1[:cut] + parent2[cut:]

    def mutate(individual):
        if random.random() < mutation_rate:
            i = random.randint(0, m - 1)
            individual[i] = random.uniform(generators[i][3], generators[i][4])

    def select(population):
        return min(population, key=fitness)

    population = [create_individual() for _ in range(population_size)]

    for _ in range(generations):
        population = [select([crossover(random.choice(population), random.choice(population)) for _ in range(population_size)]) for _ in range(population_size)]
        for individual in population:
            mutate(individual)

    best_individual = select(population)
    return best_individual, fitness(best_individual)

if __name__ == "__main__":
    import sys
    input = sys.stdin.read
    data = input().strip().split()

    tsp_n = int(data[0])
    cost_matrix = []
    index = 1
    for i in range(tsp_n):
        row = list(map(int, data[index:index + tsp_n]))
        cost_matrix.append(row)
        index += tsp_n

    tsp_route, tsp_distance = tsp_genetic_algorithm(cost_matrix)
    print(f"Route: {'->'.join(map(str, tsp_route))}")
    print(f"Distance: {tsp_distance}")

    opf_m = int(data[index])
    PD = float(data[index + 1])
    index += 2
    generators = []
    for i in range(opf_m):
        a, b, c, Pmin, Pmax = map(float, data[index:index + 5])
        generators.append((a, b, c, Pmin, Pmax))
        index += 5

    opf_outputs, opf_cost = opf_genetic_algorithm(opf_m, PD, generators)
    print(f"Generator Outputs: {', '.join(map(str, opf_outputs))}")
    print(f"Cost: {opf_cost}")
